In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sklearn.metrics
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

from pathlib import Path
import sys
pathlib = str(Path().resolve()) + "/../../"
sys.path.append(pathlib)

from utils import *
data_path = "../../data"

test_df = load_csv_to_pandasdf(os.path.join(data_path, "sharechat_recsys2023_data",  "test")).reset_index(drop=True)

0it [00:00, ?it/s]


In [2]:
def transform_count_encoded_feature(fg_list_1, df, encoder_list):
    for feature_name in tqdm(fg_list_1, desc='get_count_encoded_feature'):
        feature_name_CE = f"{feature_name}_CE"        
        encoder = encoder_list[feature_name]
        df[feature_name_CE] = encoder.transform(df[feature_name])        
        df[feature_name_CE] = df[feature_name_CE].fillna(1)
    return df

def transform_indexing_feature(fg_list_1, df, encoder_list):
    for feature_name in tqdm(fg_list_1, desc='get_indexing_feature'):
        feature_name_index = f"{feature_name}_idx"            
        encoder = encoder_list[feature_name]
        df[feature_name_index] = encoder.transform(df)
    return df

def transform_newvalue(categorical_list, df, encoder_list):
    for feature_name in tqdm(categorical_list, desc='get_newvalue_flag_feature'):        
        encoder = encoder_list[feature_name]
        df = encoder.transform(df)
    return df

In [3]:
fdflag_features = [f"f_{i}" for i in list(range(2, 23)) + [78, 75, 50]]
count_features = [f"f_{i}" for i in [2, 4, 6, 13, 15, 18] + [78, 75, 50, 20, 24]]
index_features = [f"f_{i}" for i in list(range(2, 23))]

new_value_encoder_list = {}
count_encoder_list = {}
index_encoder_list = {}
for feature_name in fdflag_features:
    new_value_encoder_list[feature_name] = NewValueEncoder.load_model(os.path.join(data_path, "1_LearningFE",  f"newvalue_{feature_name}.pkl"))

for feature_name in count_features:
    count_encoder_list[feature_name] = CountEncoder.load_model(os.path.join(data_path, "1_LearningFE",  f"count_{feature_name}.pkl"))

for feature_name in index_features:
    index_encoder_list[feature_name] = Indexer.load_model(os.path.join(data_path, "1_LearningFE",  f"index_{feature_name}.pkl"))

In [4]:
import warnings
warnings.filterwarnings("ignore")

target_label = 'is_installed'

test_df['dow'] = test_df['f_1'] % 7
test_df = transform_newvalue(fdflag_features, test_df, new_value_encoder_list)
test_df = transform_count_encoded_feature(count_features, test_df, count_encoder_list)
test_df = transform_indexing_feature(index_features, test_df, index_encoder_list)

get_indexing_feature: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:02<00:00,  9.44it/s]


In [5]:
test_df.to_parquet(os.path.join(data_path, "1_LearningFE",  "test_processed.parquet"))